In [ ]:
#이미지 폴더에서 얼굴만 잘라내서 저장
import cv2
import numpy as np
import openvino as ov
import os
core = ov.Core()
input_folder_path = 'faces'
output_folder_path = 'faces1'
face_detection_model = "models/face-detection-adas-0001.xml"
face_detection_compiled_model = core.compile_model(model=face_detection_model, device_name="CPU")
face_input_layer = face_detection_compiled_model.input(0)
face_output_layer = face_detection_compiled_model.output(0)
face_input_shape = face_input_layer.shape
image_files = [f for f in os.listdir(input_folder_path) if os.path.isfile(os.path.join(input_folder_path, f))]
image_files.sort()
def preprocess_image(image, input_shape):
    if len(image.shape) == 3 and image.shape[2] == 4:
        image = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (input_shape[3], input_shape[2]))
    image = np.transpose(image, (2, 0, 1))
    image = np.expand_dims(image, axis=0)
    image = image.astype(np.float32)
    return image
def detect_faces(image):
    preprocessed_image = preprocess_image(image, face_input_shape)
    result = face_detection_compiled_model([preprocessed_image])[face_output_layer]
    return result[0][0]
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
for image_file in image_files:
    image_path = os.path.join(input_folder_path, image_file)
    load_image = cv2.imread(image_path)
    faces = detect_faces(load_image)
    for idx, face in enumerate(faces):
        confidence = face[2]
        if confidence > 0.5:
            xmin = int(face[3] * load_image.shape[1])
            ymin = int(face[4] * load_image.shape[0])
            xmax = int(face[5] * load_image.shape[1])
            ymax = int(face[6] * load_image.shape[0])
            if xmin < 0 or ymin < 0 or xmax > load_image.shape[1] or ymax > load_image.shape[0]:
                continue
            face_image = load_image[ymin:ymax, xmin:xmax]
            output_file_path = os.path.join(output_folder_path, f"{os.path.splitext(image_file)[0]}.jpg")
            cv2.imwrite(output_file_path, face_image)
            print(f"Saved face image to {output_file_path}")